<a href="https://colab.research.google.com/github/StarNJey/localnotebook/blob/main/Local_Court_notebook_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <Chatbot 우주 by C.H.PARK>
인터넷에 연결되지 않은 컴퓨터에서도 NotebookLM과 같은 기능을 구현할 수 있을까요?

CPU만 사용할 수 있는 로컬 환경에서 실험을 해보니 결과가 괜찮아 예제를 만들어 보았습니다.

* 아래 코드는 인터넷 + colab의 클라우드 GPU를 활용합니다. 같은 결과를 GPU 없는 로컬 PC환경에서도 구현하였습니다(다만 CPU만 사용하는 특성상 아래 예제보다 속도가 훨씬 느립니다). 로컬 환경(without GPU) 구동에 관하여 궁금한 사항이 있으면 innerpeace.chp@gmail.com으로 연락주세요!

* 핵심 LLM은 LG에서 개발한 EXAONE 1.2B를 사용하였습니다. 1.2B도 성능이 꽤 괜찮아 32B를 사용하면 훨씬 더 좋은 결과가 나올 것 같습니다.

* 청크 max 크기를 크게 하고, 청크 하나에 들어가는 문장 수를 늘리면 어느 정도까지는 성능이 올라갑니다. 다만 너무 키우면 VRAM 부족으로 out of memory 에러가 발생합니다.

# 아래 부분을 실행하기 전에 다음 작업을 해야 합니다.
1. 왼쪽 상단 메뉴에서 "런타임"을 클릭합니다.
2. 펼쳐진 세부 메뉴에서 "런타임 유형 변경"을 클릭합니다.
3. "하드웨어 가속기" 중 "T4 GPU"를 선택한 후 "저장"을 누릅니다.
4. 오른쪽 상단 메뉴에서 "연결"을 클릭합니다.
5. 이제 준비가 되었습니다. 아래 부분을 실행하면, 필요한 라이브러리가 자동으로 설치됩니다.

In [ ]:
!pip install streamlit transformers sentence-transformers pdfplumber kiwipiepy

# 아래 부분을 실행하면, 제 github repository에서 미리 작성해둔 python 코드를 다운로드합니다.

In [ ]:
!git clone https://github.com/StarNJey/localnotebook.git

In [ ]:
%cd localnotebook/

# 필요한 창을 띄우기 위한 작업입니다.
1. 아래 코드를 차례대로 실행하세요.
2. **Your quick Tunnel has been created! Visit it at (it may take some time to be reachable): 다음에 나오는 링크를 클릭하면 새 창에서 프로그램이 구동됩니다.**
3. <주의점> Colab 무료 버전을 사용하는 분은 **최대 청크 길이가 400이 넘지 않도록** 세팅해야 합니다. 최대 청크 길이를 길게 하면 VRAM 부족 에러가 발생합니다. 무료 버전을 사용하는 분들은 초기 세팅을 바꾸지 않을 것을 추천드립니다.
4. <알아야 할 점> 4기가 이상의 LLM 모델을 다운로드해야 해 LLM 로드까지는 다소 시간이 소요됩니다.

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb


In [5]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!cloudflared tunnel --url http://localhost:8501

# Troubleshooting
out of memory 에러가 발생했어요!
* 최대 청크 길이를 줄이세요.
* "답변 생성" 아이콘을 한 번만 눌러주세요.
* 왼쪽 상단의 "런타임"을 클릭한 후 "세션 다시 시작 및 모두 실행"을 눌러 기다린 후 링크를 다시 클릭해서 재작업을 해주세요.